In [ ]:
import numpy as np
import pybullet as p
import time
from robot_descriptions.loaders.pybullet import load_robot_description

In [ ]:
p.connect(p.GUI_SERVER)
p.setTimeStep(0.001)
p.setGravity(0, 0, -9.81)

In [ ]:
robot_id = load_robot_description('ur5_description', useFixedBase=True) 
joint_indices = [i for i in range(p.getNumJoints(robot_id)) if p.getJointInfo(robot_id, i)[2] == p.JOINT_REVOLUTE]
for i in range(p.getNumJoints(robot_id)):
  print(p.getJointInfo(robot_id, i))

eff_idx = 7
print("End effector joint: ", eff_idx)
print("Revolute joints: ", joint_indices)

# Disable default motor control

In [ ]:
for i in joint_indices:
    p.setJointMotorControl2(
        bodyUniqueId=robot_id,
        jointIndex=i,
        controlMode=p.VELOCITY_CONTROL,
        force=0  # Disable motor forces
    )

# Feedback and Stabilization

In [ ]:
def pd_control(robot_id, q_desired, v_desired, Kp, Kd):
  joint_states = p.getJointStates(robot_id, joint_indices)
  q_measured = np.array([state[0] for state in joint_states])
  v_measured = np.array([state[1] for state in joint_states])
  position_error = q_desired - q_measured
  velocity_error = v_desired - v_measured
  
  tau = Kp * position_error + Kd * velocity_error
  # add gravity compensation
  tau += np.array(p.calculateInverseDynamics(robot_id, q_measured.tolist(), [0] * len(joint_indices), [0] * len(joint_indices)))
  return tau

Kp = 100 * np.ones_like(joint_indices)
Kd = 5 * np.ones_like(joint_indices)

def control_to_desired_configuration(robot_id, q_desired, v_desired=None, max_steps=5000):
  if v_desired is None:
    v_desired = np.zeros_like(q_desired)
  for step in range(max_steps):  # Simulation steps
    torques = pd_control(robot_id, q_desired, v_desired, Kp, Kd).tolist()
    p.setJointMotorControlArray(
      bodyUniqueId=robot_id,
      jointIndices=joint_indices,
      controlMode=p.TORQUE_CONTROL,
      forces=torques
    )
    p.stepSimulation()
    

# Test PD controller

In [ ]:
desired_positions = np.array([0.5, -1.0, 0.0, -1.5, 0.0, 1.0])
desired_velocities = np.zeros_like(desired_positions)

control_to_desired_configuration(robot_id, desired_positions, desired_velocities)

# Moving end-effector along a horizontal line

In [ ]:
# Define the start point and direction of the line
start_point = np.array([0.8, 0.3, 0.3])  # [x, y, z] in 3D space
direction_vector = np.array([0.0, -0.6, 0])  # Direction along X-axis (horizontal)
target_positions = start_point + np.linspace(0, 1, num=100)[:, np.newaxis] * direction_vector[np.newaxis, :]
print(target_positions[0], target_positions[1])
print(p.getJointInfo(robot_id, eff_idx))
# set robot at starting point
time.sleep(5)
for target in target_positions:
  joint_angles = p.calculateInverseKinematics(
    robot_id,
    eff_idx,
    target
  )
  control_to_desired_configuration(robot_id, joint_angles)
  time.sleep(1 / 240.0)